In [15]:
import json
import os
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [2]:
class DataLoader:
    def __init__(self,datasetPath,batch,imageSegmentNo) -> None:
        self.datasetPath = datasetPath
        
        self.batchPath = f'{datasetPath}/{batch}'
        self.classes = os.listdir(self.batchPath)


        self.imageSegmentNo = imageSegmentNo

        self.scaledData = []
        

        self.dataExtractor()
        self.shuffler()
        

    def dataExtractor(self):
        for i in range(len(self.classes)):
            if len(self.classes[i].split('.'))<=1:

                for imgdir in os.listdir(f'{self.batchPath}/{self.classes[i]}'):
                    if len(imgdir.split('.'))<=1:
                        jsonMetadata = self.jsonDataLoader(f'{self.batchPath}/{self.classes[i]}/{imgdir}/metadata.json')
                        

                        self.scaledData.append([  self.featureScaler(jsonMetadata['features'][self.imageSegmentNo]), jsonMetadata['classes'][self.imageSegmentNo]])
                        # self.y.append()


                        


    def jsonDataLoader(self,jsonPath):
        with open(jsonPath) as jsonfile:
            currentData = json.load(jsonfile)
            return currentData

    def featureScaler(self,feature):

        maxVal = max(feature[0])
        minVal = min(feature[0])

        dezmin = 0
        dezmax = 1

        scaledFeatures =  []
        for i in feature[0]:
            scaled = (    (   (i - minVal)/(maxVal-minVal)  ) /   (dezmax-dezmin)    ) + dezmin
            scaledFeatures.append(scaled)

        return np.array(scaledFeatures)


    def shuffler(self):
        valData=  np.array(self.scaledData)
        np.random.shuffle(valData)
        
        ListX = []
        Listy = []


        for i in valData:
            ListX.append(i[0])
            Listy.append(i[1])

        self.X = np.array(ListX)
        self.y = np.array(Listy)
        


In [19]:
valDataComplete = []
for i in range(9):
    valDataComplete.append(DataLoader('All_200X','val',i))


/var/folders/4m/_7y4kbtd0l5gjj3fhdysn97c0000gn/T/ipykernel_15246/3157486618.py:56: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  valData=  np.array(self.scaledData)


In [20]:
trainDataComplete = []
for i in range(9):
    trainDataComplete.append(DataLoader('All_200X','train',i))

/var/folders/4m/_7y4kbtd0l5gjj3fhdysn97c0000gn/T/ipykernel_15246/3157486618.py:56: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  valData=  np.array(self.scaledData)


In [21]:
testDataComplete = []
for i in range(9):
    testDataComplete.append(DataLoader('All_200X','test',i))

/var/folders/4m/_7y4kbtd0l5gjj3fhdysn97c0000gn/T/ipykernel_15246/3157486618.py:56: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  valData=  np.array(self.scaledData)


In [23]:
clf1 = SVC(kernel='rbf', C=1)
clf1.fit(trainDataComplete[0].X,trainDataComplete[0].y)

y_pred1 = clf1.predict(testDataComplete[0].X)
print(accuracy_score(testDataComplete[0].y,y_pred1))

clf2 = SVC(kernel='rbf', C=1)
clf2.fit(trainDataComplete[1].X,trainDataComplete[1].y)

y_pred2 = clf2.predict(testDataComplete[1].X)
print(accuracy_score(testDataComplete[1].y,y_pred2))

clf3 = SVC(kernel='rbf', C=1)
clf3.fit(trainDataComplete[2].X,trainDataComplete[2].y)

y_pred3 = clf3.predict(testDataComplete[2].X)
print(accuracy_score(testDataComplete[2].y,y_pred3))


clf4 = SVC(kernel='rbf', C=1)
clf4.fit(trainDataComplete[3].X,trainDataComplete[3].y)

y_pred4 = clf4.predict(testDataComplete[3].X)
print(accuracy_score(testDataComplete[3].y,y_pred4))

clf5 = SVC(kernel='rbf', C=1)
clf5.fit(trainDataComplete[4].X,trainDataComplete[4].y)

y_pred5 = clf5.predict(testDataComplete[4].X)
print(accuracy_score(testDataComplete[4].y,y_pred5))


clf6 = SVC(kernel='rbf', C=1)
clf6.fit(trainDataComplete[5].X,trainDataComplete[5].y)

y_pred6 = clf6.predict(testDataComplete[5].X)
print(accuracy_score(testDataComplete[5].y,y_pred6))


clf7 = SVC(kernel='rbf', C=1)
clf7.fit(trainDataComplete[6].X,trainDataComplete[6].y)

y_pred7 = clf7.predict(testDataComplete[6].X)
print(accuracy_score(testDataComplete[6].y,y_pred7))


clf8 = SVC(kernel='rbf', C=1)
clf8.fit(trainDataComplete[7].X,trainDataComplete[7].y)

y_pred8 = clf8.predict(testDataComplete[7].X)
print(accuracy_score(testDataComplete[7].y,y_pred8))


clf9 = SVC(kernel='rbf', C=1)
clf9.fit(trainDataComplete[8].X,trainDataComplete[8].y)

y_pred9 = clf9.predict(testDataComplete[8].X)
print(accuracy_score(testDataComplete[8].y,y_pred9))



0.8360655737704918
0.8426229508196721
0.8426229508196721
0.8557377049180328
0.8360655737704918
0.8557377049180328
0.8426229508196721
0.8327868852459016
0.8426229508196721
